<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Техническое-задание-для-построения-дашборда" data-toc-modified-id="Техническое-задание-для-построения-дашборда-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Техническое задание для построения дашборда</a></span></li><li><span><a href="#Структура-датафрейма" data-toc-modified-id="Структура-датафрейма-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Структура датафрейма</a></span></li><li><span><a href="#Подсоединение-к-базе-данных" data-toc-modified-id="Подсоединение-к-базе-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подсоединение к базе данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Изучение-датафрейма" data-toc-modified-id="Изучение-датафрейма-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Изучение датафрейма</a></span></li><li><span><a href="#Получение-csv-файла" data-toc-modified-id="Получение-csv-файла-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Получение csv-файла</a></span></li><li><span><a href="#Дашборд" data-toc-modified-id="Дашборд-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Дашборд</a></span></li><li><span><a href="#Общие-выводы" data-toc-modified-id="Общие-выводы-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Общие выводы</a></span></li></ul></div>

# Анализ взаимодействия пользователей с карточками Яндекс.Дзен

## Техническое задание для построения дашборда

Бизнес-задача:
анализ взаимодействия пользователей с карточками Яндекс.Дзен;

Насколько часто предполагается пользоваться дашбордом: 
не реже, чем раз в неделю;

Кто будет основным пользователем дашборда: 
менеджеры по анализу контента;

Состав данных для дашборда:
1. История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
2. Разбивка событий по темам источников;
3. Таблица соответствия тем источников темам карточек;

По каким параметрам данные должны группироваться:
1. Дата и время;
2. Тема карточки;
3. Тема источника;
4. Возрастная группа;

Характер данных:
История событий по темам карточек — абсолютные величины с разбивкой по минутам;
Разбивка событий по темам источников — относительные величины (% событий);
Соответствия тем источников темам карточек - абсолютные величины;
Важность: все графики имеют равную важность;

## Структура датафрейма

`record_id` — первичный ключ,
`item_topic` — тема карточки,
`source_topic` — тема источника,
`age_segment` — возрастной сегмент,
`dt` — дата и время,
`visits` — количество событий.

## Подсоединение к базе данных

In [1]:
# Установка недостающего модуля
!pip install psycopg2-binary

In [2]:
# Установка библиотек:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Создание коннекции к базе данных:

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

In [4]:
# Создание sql-запроса

query = '''
           SELECT *
           FROM dash_visits
        '''
# Сохраняем sql-запрос в переменной dash_visits
dash_visits = pd.io.sql.read_sql(query, con=engine)

In [5]:
# Изучение датафрейма
dash_visits.head(10)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
8,1040605,Деньги,Авто,18-25,2019-09-24 19:00:00,5
9,1040606,Деньги,Авто,26-30,2019-09-24 18:29:00,1


## Предобработка данных

In [6]:
# Смотрим информацию по датафрейму
dash_visits.info()
# Пустых строк нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


В датафрейме 6 столбцов и 30745 строк. Тип данных изменять не требуется.

In [7]:
# Избавление от экспоненты
pd.options.display.float_format = '{:,.2f}'.format
# Основная статистика датафрейма
dash_visits.describe()

,record_id,visits
count,"30,745.00","30,745.00"
mean,"1,055,969.00",10.09
std,"8,875.46",19.73
min,"1,040,597.00",1.00
25%,"1,048,283.00",1.00
50%,"1,055,969.00",3.00
75%,"1,063,655.00",10.00
max,"1,071,341.00",371.00


Аномальных значений не выявлено.

In [8]:
# Проверка на явные дубликаты
dash_visits['record_id'].duplicated().sum()

0

Дубликаты отсутствуют.

## Изучение датафрейма

In [9]:
# Темы публикаций и их количество:
dash_visits['item_topic'].value_counts()

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

Огромный спектр тем публикаций. Больше всего - по темам "Отношения", "Интересные факты" и "Наука".

In [10]:
# Темы источников и их количество:
dash_visits['source_topic'].value_counts()

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

Основные источники публикаций - Семейные отношения, Россия и Знаменитости

In [11]:
# Возраста и их количество
dash_visits['age_segment'].value_counts()

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

Основной возрастной сегмент - 18-25 лет.

In [12]:
# Временной интервал в датафрейме
print(dash_visits['dt'].min(), '|', dash_visits['dt'].max())

2019-09-24 18:28:00 | 2019-09-24 19:00:00


Данные получены за 32-минутный промежуток времени.

## Получение csv-файла

In [13]:
# Преобразование в csv
dash_visits.to_csv('dash_visits.csv', index = False)

dash_visits.head(5)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


## Дашборд

Доступ по ссылке: https://public.tableau.com/views/MyprojectViz_Yandex_Dzen/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link

## Общие выводы

Презентация доступна по ссылке:https://disk.yandex.ru/i/1Jak3RH9sw07LA